## 8-1 perceptron

    - 단층 퍼셉트론의 한계 : xor 불가능
    - xor training
    
   ![title](pe.png)

In [1]:
import torch

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

- xor : 2개의 입력값이 다를 때, 출력이 1

In [3]:
X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]]).to(device)
Y = torch.FloatTensor([[0], [1], [1], [0]]).to(device)

In [4]:
# 단층, 입력 2, 출력 1
linear = torch.nn.Linear(2, 1, bias=True)
sigmoid = torch.nn.Sigmoid()

- Sequential : 코드에 적힌 순서대로 값을 전달해 처리한다.

In [5]:
model = torch.nn.Sequential(linear, sigmoid).to(device)

- BCELoss : Binary Cross Entropy
- SGD : 확률적 경사 하강법

In [6]:
criterion = torch.nn.BCELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1)

In [7]:
from tqdm import tqdm_notebook as tqdm

In [8]:
for step in tqdm(range(10001)):
    
    optimizer.zero_grad()
    hypothesis = model(X)

    # cost function, GD 개선
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    if step % 1000 == 0:
        #with torch.no_grad():
        hypothesis = model(X)
        predicted = (hypothesis > 0.5).float()
        accuracy = (predicted == Y).float().mean()
            
        print(step, "\t" ,cost.item(), "\t", accuracy.item())

0 	 0.7273974418640137 	 0.75
1000 	 0.6931471824645996 	 0.5
2000 	 0.6931471824645996 	 0.5
3000 	 0.6931471824645996 	 0.5
4000 	 0.6931471824645996 	 0.5
5000 	 0.6931471824645996 	 0.5
6000 	 0.6931471824645996 	 0.5
7000 	 0.6931471824645996 	 0.5
8000 	 0.6931471824645996 	 0.5
9000 	 0.6931471824645996 	 0.5
10000 	 0.6931471824645996 	 0.5



- True if hypothesis>0.5 
- else False

In [9]:
with torch.no_grad():
    hypothesis = model(X)
    predicted = (hypothesis > 0.5).float()
    accuracy = (predicted == Y).float().mean()
    print('\nHypothesis: ', hypothesis.detach().cpu().numpy(), '\nCorrect: ', predicted.detach().cpu().numpy(), '\nAccuracy: ', accuracy.item())


Hypothesis:  [[0.5]
 [0.5]
 [0.5]
 [0.5]] 
Correct:  [[0.]
 [0.]
 [0.]
 [0.]] 
Accuracy:  0.5


## 8-2 Multi Layer perceptron

- multi layer 
    - F 대신에 변수명을 사용하여 함수처럼 사용

In [28]:
X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]]).to(device)
Y = torch.FloatTensor([[0], [1], [1], [0]]).to(device)

![title](xor.png)

In [29]:
linear1 = torch.nn.Linear(2, 2, bias=True)
linear2 = torch.nn.Linear(2, 1, bias=True)
sigmoid = torch.nn.Sigmoid()

In [30]:
model = torch.nn.Sequential(linear1, sigmoid, linear2, sigmoid).to(device)

- binary cross entropy, stochastic gradient descent

In [31]:
criterion = torch.nn.BCELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1)

In [32]:
for step in tqdm(range(10001)):
    optimizer.zero_grad()
    hypothesis = model(X)

    # cost/loss function
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    if step % 1000 == 0:
        hypothesis = model(X)
        predicted = (hypothesis > 0.5).float()
        accuracy = (predicted == Y).float().mean()
        
        print(step, "\t",cost.item(), "\t", accuracy.item())

0 	 0.6935256719589233 	 0.25
1000 	 0.32183659076690674 	 1.0
2000 	 0.010077964514493942 	 1.0
3000 	 0.004875706974416971 	 1.0
4000 	 0.003201704705134034 	 1.0
5000 	 0.0023797699250280857 	 1.0
6000 	 0.0018922294257208705 	 1.0
7000 	 0.001569831627421081 	 1.0
8000 	 0.0013409029925242066 	 1.0
9000 	 0.001170062692835927 	 1.0
10000 	 0.0010377232683822513 	 1.0



In [33]:
with torch.no_grad():
    hypothesis = model(X)
    predicted = (hypothesis > 0.5).float()
    accuracy = (predicted == Y).float().mean()
    print('\nHypothesis: ', hypothesis.detach().cpu().numpy(), '\nCorrect: ', predicted.detach().cpu().numpy(), '\nAccuracy: ', accuracy.item())


Hypothesis:  [[9.9353190e-04]
 [9.9907660e-01]
 [9.9863094e-01]
 [8.6218247e-04]] 
Correct:  [[0.]
 [1.]
 [1.]
 [0.]] 
Accuracy:  1.0


- learning rate 에 따른 cost function 비교

In [16]:
learning = [0.1, 0.3, 0.5, 0.7, 1.0]

for i in learning :
    # learning list에 들어있는 값으로 변경
    optimizer2 = torch.optim.SGD(model.parameters(), lr=i)
    # learning rate 바뀔때마다 데이터 초기화
    X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]]).to(device)
    Y = torch.FloatTensor([[0], [1], [1], [0]]).to(device)
    for step in tqdm(range(10001)):
        optimizer2.zero_grad()
        hypothesis = model(X)

        # cost/loss function
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer2.step()

        # learning rate 마다 최종 cost 값 출력
        if step == 10000:
            print(i, "\t",cost.item())

0.1 	 0.3471031188964844



0.3 	 0.34697872400283813



0.5 	 0.3468639850616455



0.7 	 0.3467811942100525



1.0 	 0.34672045707702637



## 8-3 nn wide deep
    - 3층 이상

In [17]:
# 데이터셋 초기화
X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]]).to(device)
Y = torch.FloatTensor([[0], [1], [1], [0]]).to(device)

In [18]:
linear1 = torch.nn.Linear(2, 10, bias=True)
linear2 = torch.nn.Linear(10, 10, bias=True)
linear3 = torch.nn.Linear(10, 10, bias=True)
linear4 = torch.nn.Linear(10, 1, bias=True)
sigmoid = torch.nn.Sigmoid()

In [19]:
model = torch.nn.Sequential(linear1, sigmoid, linear2, sigmoid, linear3, sigmoid, linear4, sigmoid).to(device)

In [20]:
criterion = torch.nn.BCELoss().to(device)
optimizer3 = torch.optim.SGD(model.parameters(), lr=1)

In [22]:
for step in tqdm(range(10001)):
    optimizer3.zero_grad()
    hypothesis = model(X)

    # cost/loss function
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer3.step()

    if step % 1000 == 0:
        hypothesis = model(X)
        predicted = (hypothesis > 0.5).float()
        accuracy = (predicted == Y).float().mean()
        
        print(step,"\t", "%.9f"%cost.item(), "\t", accuracy.item())

0 	 0.694487214 	 0.5
1000 	 0.693120003 	 0.75
2000 	 0.693009973 	 0.75
3000 	 0.691773057 	 0.75
4000 	 0.002656406 	 1.0
5000 	 0.000615512 	 1.0
6000 	 0.000333397 	 1.0
7000 	 0.000225392 	 1.0
8000 	 0.000169128 	 1.0
9000 	 0.000134746 	 1.0
10000 	 0.000111676 	 1.0



In [23]:
with torch.no_grad():
    hypothesis = model(X)
    predicted = (hypothesis > 0.5).float()
    accuracy = (predicted == Y).float().mean()
    print('\nHypothesis: ', hypothesis.detach().cpu().numpy(), '\nCorrect: ', predicted.detach().cpu().numpy(), '\nAccuracy: ', accuracy.item())


Hypothesis:  [[8.2612169e-05]
 [9.9989307e-01]
 [9.9988806e-01]
 [1.4519275e-04]] 
Correct:  [[0.]
 [1.]
 [1.]
 [0.]] 
Accuracy:  1.0


In [26]:
learning = [0.1, 0.3, 0.5, 0.7, 1.0]

for i in learning :
    # learning list에 들어있는 값으로 변경
    optimizer4 = torch.optim.SGD(model.parameters(), lr=i)
    # learning rate 바뀔때마다 데이터 초기화
    X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]]).to(device)
    Y = torch.FloatTensor([[0], [1], [1], [0]]).to(device)
    for step in tqdm(range(10001)):
        optimizer4.zero_grad()
        hypothesis = model(X)

        # cost/loss function
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer4.step()

        # learning rate 마다 최종 cost 값 출력
        if step == 10000:
            # '%.10f' % num
            print(i, "\t",'%.10f' %cost.item())

0.1 	 0.0000345266



0.3 	 0.0000294302



0.5 	 0.0000235590



0.7 	 0.0000183435



1.0 	 0.0000139178

